# Guía 0 - Análisis exploratorio de datos
## Descarga y exploración de una DB para el análisis de sus datos

En general, toda búsqueda de información o proceso de descubrimiento comienza con un problema o una pregunta. Usualmente, también traemos ideas previas o tentativas que de alguna manera trataremos de confirmar, ampliar o rectificar. En resumen, el análisis de datos nos exige antes comprender claramente que representan, conocer cómo se generaron y qué reglas aplican para luego poder interpretarlos y avanzar en modelos de predicción o pronóstico.

Actualmente, la oferta de datos es inmensa dado el progreso y abaratamiento de los instrumentos de medición, la mayor capacidad de almacenamiento y comunicación. Asimismo, las políticas de datos abiertos (Open Data) alentadas por la comunidad científica, asimiladas por muchas empresas y sostenidas por las iniciativas de Gobierno Abierto han contribuído a la abundancia de datos y su interés por la creación de más valor a través de su reutilización.

## Estudio de caso: serie 2017-2025 de demanda eléctrica diaria en Argentina 

Dentro de este contexto tecnológico e institucional y a modo de ejemplo, tomaremos los datos publicados por Cammesa para ejercitarnos en las técnicas de análisis y modelado de series temporales que iremos incorporando en sucesivos estudios de casos. La Compañía Administradora del Mercado Mayorista Eléctrico Sociedad Anónima (CAMMESA) coordina las operaciones de despacho, la responsabilidad por el establecimiento de los precios mayoristas y la administración de las transacciones económicas que se realizan a través del Sistema Argentino de Interconexión (SADI). 

Captura de pantalla del sitio web de Cammesa: 
![Captura de pantalla del sitio web de Cammesa](Figs/cammesa_sitio_web_2026.png)

La propuesta es descragar la DB que contiene información histórica entre los años 2017 y 2025 para la demanda neta eléctrica diaria en Argentina.

Captura de pantalla del sitio web de Cammesa con el enlace para la [descarga](https://cammesaweb.cammesa.com/2026/01/05/covid-19-comportamiento-de-la-demanda-de-energia-electrica-en-el-mem/#) de las DB: 
![Captura de pantalla del sitio de descarga de los datos de Cammesa](Figs/cammesa_db_2017_2025.png)

Luego de descomprimir el archivo podemos desplegar la planilla que contiene diferentes solapas. Particularmente, estamos interesados en la solapa `Datos Región`. 

Captura de pantalla del contenido del archivo `'Base Demanda Diaria 2017 2025.xlsx` desplegado en una hoja de cálculo: 
![Captura de pantalla de la hoja de cálculo desplegando la planilla](Figs/cammesa_planilla_2017_2023.png) 


### Primer contacto con los datos

Comnezamos cargando la hoja con los datos de interés y visualizamos sus primeros registros


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

baseDiaria = pd.read_excel('Data/Base Demanda Diaria 2017 2025.xlsx',
                           sheet_name='Datos Región', skiprows=4)

baseDiaria.head()

Conocemos el nombre o encabezado de las columnas y profundizamos en el tipo de dato que almacenan al igual que su tamaño.

In [ ]:
baseDiaria.info()

Es interesante notar que la columna fecha ya se detectó y codificó correctamente como tipo de dato `datetime64[ns]`. Si bien esta *característica o feature* permite conocer el día de la semana,

In [ ]:
baseDiaria['Fecha'].dt.day_name()

es trabajoso recuperar los días feriados. Por eso, nos quedamos con la columna `Tipo día`

In [ ]:
baseDiaria['Tipo día'].value_counts()

En adelante, nos limitamos a los atributos que usaremos en el estudio de la demanda eléctrica. Además, definimos el índice para facilitar los cálculos de distancias entre fechas

In [ ]:
baseDiaria.set_index(pd.to_datetime(baseDiaria['Fecha']),inplace=True)
baseDiaria = baseDiaria[['DEMANDA TOTAL', 'Tipo día', 'TEMPERATURA REFERENCIA MEDIA GBA °C']]
baseDiaria.rename(columns={'TEMPERATURA REFERENCIA MEDIA GBA °C':'TEMP MEDIA'}, inplace=True)
baseDiaria['Interval'] = np.float32((baseDiaria.index - baseDiaria.index[0]).days)

**Ejercicios**
1. Confirmar que no faltan fechas ni valores en la base de datos diaria.
2. Buscar el mínimo de demanda eléctrica total y **Googlear que pasó ese día**.
3. Representar `DEMANDA TOTAL` en función de la `Fecha`. Observar la presencia de un patrón de mínimos y máximos que se repiten todos los años.
4. Representar `DEMANDA TOTAL` en función de `TEMP MEDIA`.
5. Realizar un *boxplot* de `DEMANDA TOTAL` según `Tipo día`.
6. Proponer modelos *benchmarks* que estimen la 'DEMANDA TOTAL'. Se busca fijar un nivel de acuerdo mínimo contra el cual comparar a los próximos modelos más avanzados. Obtener métricas como el coeficiente de determinación, MAE, RMSE, entre otras opciones.  
    a. Un modelo que siempre predice la media de la serie.  
    b. Un modelo que siempre predice el valor del día anterior.  
    c. Un modelo que siempre predice el valor del mismo día de la semana anterior.


**Respuestas**

Al utilizar el método `asfreq()` en `Pandas`, cualquier espacio en la serie temporal se rellenará con valores NaN para que coincidan con la frecuencia especificada. Para comprobar si hay valores faltantes que puedan producirse después de esta transformación

In [ ]:
baseDiaria.asfreq('D')
# Missing values
# ==============================================================================
print(f'Number of rows with missing values: {baseDiaria.isna().any().sum()}')

Aunque no es necesario, ya que se ha establecido una frecuencia, es posible verificar que la serie temporal está completa.

In [ ]:
# Verify that a temporary index is complete
# ==============================================================================
start_date = baseDiaria.index.min()
end_date = baseDiaria.index.max()
complete_date_range = pd.date_range(start=start_date, end=end_date, freq=baseDiaria.index.freq)
is_index_complete = (baseDiaria.index == complete_date_range).all()
print(f"Index complete: {is_index_complete}")

In [ ]:
import matplotlib.pyplot as plt
baseDiaria['DEMANDA TOTAL'].plot()
plt.ylabel('DEMANDA TOTAL (MW)')

In [ ]:
import seaborn as sns

sns.relplot(
    data=baseDiaria,
    x='TEMP MEDIA', y='DEMANDA TOTAL', hue='Tipo día'
)

In [ ]:
sns.boxplot(
    data=baseDiaria,
    x='Tipo día', y='DEMANDA TOTAL', formatter=(lambda x: x.split()[0])
)

In [ ]:
print('a. Modelo que predice la media')
print(f'La media es {np.mean(baseDiaria['DEMANDA TOTAL']):.0f}MW')
print(f'La varianza para el modelo que predice la media es {np.sqrt(np.var(baseDiaria['DEMANDA TOTAL'])):.0f}MW')

print('b. Modelo que predice repitiendo el valor del día anterior')
print(f'La varianza para el residuo del modelo que repite el valor de ayer es  \
{np.sqrt(np.var(baseDiaria['DEMANDA TOTAL']- baseDiaria['DEMANDA TOTAL'].shift(1))):.0f}MW')